In [30]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from path import Path

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F


class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
    
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud

class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform if not valid else default_transforms()
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}

train_transforms = transforms.Compose([
                    PointSampler(1024),
                    Normalize(),
                    RandRotation_z(),
                    RandomNoise(),
                    ToTensor()
                    ])



class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix


class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 100):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64


def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

def load_test_data(path,c, category, epoch, example):
    if len(str(epoch))>4:
      n= str(epoch)
    else:
      n= '0'*(3-len(str(epoch)))+str(epoch)
    y= '0'*(3-len(str(category)))+str(category)
    #path+ bathtub/bathtub_000/coordinates/bathtub_000_300_2_recon_coord.pt
    coordinates = torch.load(path+c+'/'+c+'_'+y+'/coordinates/'+c+'_'+y+'_'+n+'_'+str(example)+'_'+'recon_coord.pt')
    coordinates= coordinates.T
    coordinates= Normalize()(coordinates)
    sampled_coordinates= torch.from_numpy(np.asarray([coordinates[np.random.choice(len(coordinates), size=1024, replace=False)]]))
    labels= torch.from_numpy(np.asarray([int(category)]))
    return sampled_coordinates, labels


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)
pointnet = PointNet()
pointnet.to(device);

optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

pointnet = PointNet()
pointnet.load_state_dict(torch.load('/home/ubuntu/Pointnet/trained/July03/experiment_01/interclass/save_39.pth'))
pointnet.eval();

cuda:0


RuntimeError: Error(s) in loading state_dict for PointNet:
	size mismatch for fc3.weight: copying a param with shape torch.Size([10, 256]) from checkpoint, the shape in current model is torch.Size([100, 256]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([100]).

In [25]:
def read_pt(filepath):
    #print(filepath)
    full_pointcloud = torch.load(filepath)
    full_pointcloud= full_pointcloud.T
    return full_pointcloud

def getDirNum(num):
    l= len(str(num))
    return '0'*(3-l) +str(num)

def load_orig_data(c, category):    
    folder_path=  '/home/ubuntu/3d-point-clouds-HyperCloud/3d_point_cloud/dataset/shapenet_training_no_noise/'

    filepath= folder_path+c+'/'+c+'_'+getDirNum(category)+'/'+c+'_'+getDirNum(category)+'.pt'
    pointcloud= read_pt(filepath).T
    coordinates= Normalize()(pointcloud)
    sampled_coordinates= torch.from_numpy(np.asarray([coordinates[np.random.choice(len(coordinates), size=1024, replace=False)]]))
    #coordinates[np.random.choice(len(coordinates), size=1024, replace=True)]
    labels= torch.from_numpy(np.asarray([int(category)]))
    
    return sampled_coordinates, labels

In [28]:
classesy=['bathtub',  'bed',  'bench',  'bookshelf',  'cabinet',  'chair',  'lamp',  'monitor',  'sofa',  'table']

results=np.zeros((10,100,10))
count=0
for c in classesy:
    path= '/home/ubuntu/3d-point-clouds-HyperCloud/experiments_arpit/all_intra_class_multiple_epoch_dataset100/'
    print (c)
    with torch.no_grad():
        for category in range(100): 
            #print('Class '+str(category))
            inputs, labels = load_orig_data(c, category)
            outputs, __, __ = pointnet(inputs.transpose(1,2))
            results[count][category]= outputs 
    count+=1
    
np.save('/home/ubuntu/Pointnet/results/July03/interclass_baseline.npy', results)



bathtub
bed
bench
bookshelf
cabinet
chair
lamp
monitor
sofa
table


In [31]:
classesy=['bathtub',  'bed',  'bench',  'bookshelf',  'cabinet',  'chair',  'lamp',  'monitor',  'sofa',  'table']
results=np.zeros((10,100,100))
count=0
for c in classesy:
    pointnet = PointNet()
    pointnet.load_state_dict(torch.load('/home/ubuntu/Pointnet/trained/July03/experiment_01/intraclass/'+c+'/save_49.pth'))
    pointnet.eval();
    print (c)
    with torch.no_grad():
        for category in range(100): 
            #print('Class '+str(category))
            inputs, labels = load_orig_data(c, category)
            outputs, __, __ = pointnet(inputs.transpose(1,2))
            results[count][category]= outputs 
    count+=1
    
np.save('/home/ubuntu/Pointnet/results/July03/intraclass_baseline.npy', results)

bathtub
bed
bench
bookshelf
cabinet
chair
lamp
monitor
sofa
table


In [18]:
path= '/home/ubuntu/3d-point-clouds-HyperCloud/experiments_arpit/all_intra_class_multiple_epoch_dataset100/'
inputs, labels = load_test_data(path,'chair', 0, 100,0)

In [19]:
inputs.shape

torch.Size([1, 1024, 3])

In [26]:
inputs, labels = load_orig_data('chair', 0)
inputs.shape

torch.Size([1, 1024, 3])